Empirical Bayes

Import necessary libraries

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import norm, boxcox
from scipy import stats
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

random_seed = 42

In [34]:
data_set = 'red'
data_set = 'white'
df = pd.read_csv("winequality-"+data_set+".csv", sep=";")
#df = df.drop_duplicates()

### Preprocessing data

#### Removing outliers

In [35]:
cols = ['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

#### Check skewness

In [36]:
df["fixed acidity"], _ = boxcox(df["fixed acidity"])
df["residual sugar"], _ = boxcox(df["residual sugar"])
df["free sulfur dioxide"], _ = boxcox(df["free sulfur dioxide"])    
df["total sulfur dioxide"], _ = boxcox(df["total sulfur dioxide"])
df["alcohol"], _ = boxcox(df["alcohol"])

#### Redefining classes

We divide the target variable (quality) into 3 classes.
- Class 0 : Bad (quality <= 4)
- Class 1 : Average (5 <= quality <= 7)
- Class 2 : Good (8 <= quality)

In [37]:
X = df.drop("quality", axis=1)

classes = []
for i in df['quality']:
    if i<=4:
        classes.append(1)
    elif i >= 5 and i <= 7:
        classes.append(2)
    elif i >= 8:
        classes.append(3)
df['classes'] = classes

Counter(df['classes'])

Counter({2: 3769, 3: 149, 1: 97})

#### Standardisation
The features are not directly comparable as they represent different physical quantities, we need to standardise them.

In [38]:
sc = StandardScaler()
X = sc.fit_transform(X)

#### Principal Components Analysis

In [39]:
pca = PCA(n_components=8)
X = pca.fit_transform(X)

#### Split the data in train and test sets

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.25, random_state=random_seed)

#### SMOTE

In [41]:
sm = SMOTE(random_state=random_seed)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [42]:
Counter(y_train)

Counter({2: 2827, 3: 2827, 1: 2827})

### Training

#### Optimal Bayes

In [43]:
from scipy.stats import norm

class BayesOptimalClassifier:
    def __init__(self):
        self.class_priors = {}
        self.class_conditional_dists = {}

    def fit(self, X, y):
        """Train the classifier by computing priors and likelihoods"""
        n_samples, _ = X.shape
        classes = np.unique(y)

        # Compute class priors P(C_k)
        class_counts = Counter(y)
        self.class_priors = {c: class_counts[c] / n_samples for c in classes}

        # Compute P(X | C_k) assuming Gaussian distributions for each feature
        self.class_conditional_dists = {}
        for c in classes:
            X_c = X[y == c]
            self.class_conditional_dists[c] = {
                "mean": X_c.mean(axis=0),
                "std": X_c.std(axis=0, ddof=1) + 1e-9  # Avoid division by zero
            }

    def predict_proba(self, X):
        """Compute posterior probabilities for each class"""
        posteriors = []
        for x in X:
            class_probs = {}
            for c in self.class_priors:
                prior = self.class_priors[c]
                likelihood = np.prod(norm.pdf(x, self.class_conditional_dists[c]["mean"], 
                                              self.class_conditional_dists[c]["std"]))
                class_probs[c] = prior * likelihood
            # Normalize probabilities
            total = sum(class_probs.values())
            posteriors.append({c: class_probs[c] / total for c in class_probs})
        return posteriors

    def predict(self, X):
        """Predict the most probable class for each instance"""
        posteriors = self.predict_proba(X)
        return np.array([max(p, key=p.get) for p in posteriors])

In [ ]:
model = BayesOptimalClassifier()
model.fit(X_train, y_train)

### Evaluation

In [45]:
y_pred = model.predict(X_test)
print('\n',classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))



               precision    recall  f1-score   support

           1       0.08      0.65      0.14        26
           2       0.97      0.50      0.66       942
           3       0.07      0.61      0.13        36

    accuracy                           0.50      1004
   macro avg       0.37      0.59      0.31      1004
weighted avg       0.91      0.50      0.62      1004

Accuracy:  0.5049800796812749
